# Multi-Class Classification using Neural Network with 3 Hidden Layers

**Assignment Objective:** Build a 3-hidden layer neural network from scratch for multi-class classification, implement forward and backward propagation, and evaluate performance on synthetic data.

## Network Architecture
- **Input Layer:** 2 features
- **Hidden Layer 1:** 64 neurons (ReLU activation)
- **Hidden Layer 2:** 32 neurons (ReLU activation)
- **Hidden Layer 3:** 16 neurons (ReLU activation)
- **Output Layer:** 5 classes (Softmax activation)
- **Loss Function:** Cross-Entropy Loss
- **Optimizer:** Gradient Descent with Learning Rate = 0.01

## 1. Import Libraries and Generate Dataset

Generate a synthetic 5-class dataset using sklearn's `make_blobs` function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic dataset
n_samples = 1500
n_classes = 5
n_features = 2

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=n_classes, cluster_std=1.5, random_state=42)

print(f"Dataset shape: {X.shape}")
print(f"Number of classes: {n_classes}")
print(f"Class distribution: {np.bincount(y)}")